Исходя из эмбединга пользователя и фильма построить рекомендательную нейросетевую систему для пользователя. Критерием выбора будет рейтинг фильма, который пользователь поставит фильму и предсказанный системой - нулевая ошибка является верной работой системы.

Список фильмов предсказанных системой потом сортируется и выдает пользователю топ-5

# Import libraries

In [1]:
import pandas as pd
import numpy as np

import os

# Import data

In [2]:
pwd!

'/home/gishb/PycharmProjects/VK_test_ml'

In [3]:
os.listdir('.')[6]

'raw_data'

In [4]:
link = os.listdir('.')[6]

In [5]:
movies_df = pd.read_csv(link+'/movies.csv')
rating_df = pd.read_csv(link+'/ratings.csv')
# links_df = pd.read_csv('links.csv')
tags_df = pd.read_csv(link+'/tags.csv')

# Data description

In [6]:
tags_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1108997 entries, 0 to 1108996
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   userId     1108997 non-null  int64 
 1   movieId    1108997 non-null  int64 
 2   tag        1108981 non-null  object
 3   timestamp  1108997 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 33.8+ MB


In [7]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58098 entries, 0 to 58097
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  58098 non-null  int64 
 1   title    58098 non-null  object
 2   genres   58098 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.3+ MB


In [8]:
rating_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27753444 entries, 0 to 27753443
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 847.0 MB


# Simple data check

In [9]:
rating_df = rating_df.dropna(how='any',axis=0)
movies_df = movies_df.dropna(how='any',axis=0)
tags_df = tags_df.dropna(how='any', axis=0)

# Create one dataframe to start workflow

In [10]:
df_general_test_0 = pd.merge(movies_df, tags_df, on="movieId")
df_general_test_0.head()

,movieId,title,genres,userId,tag,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1040,animated,1514919574
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1040,buddy movie,1514919577
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1040,Cartoon,1514919589
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1040,cgi,1514919556
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1040,comedy,1514919569


In [11]:
general_df = pd.merge(df_general_test_0, rating_df, how='left', left_on=['movieId','userId'], right_on = ['movieId','userId'])

In [12]:
general_df["genres"] = general_df["genres"].str.split('|')

In [13]:
general_df.head()

,movieId,title,genres,userId,tag,timestamp_x,rating,timestamp_y
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1040,animated,1514919574,2.5,1.514920e+09
1,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1040,buddy movie,1514919577,2.5,1.514920e+09
2,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1040,Cartoon,1514919589,2.5,1.514920e+09
3,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1040,cgi,1514919556,2.5,1.514920e+09
4,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1040,comedy,1514919569,2.5,1.514920e+09


# Create user data

In [14]:
import itertools
user_data = pd.DataFrame()

In [15]:
user_tags_raw = general_df.groupby('userId')['tag'].apply(set).apply(list).reset_index() #каждый user_id имеет свой список tags
user_data['userId'] = user_tags_raw['userId']
user_data['user_all_tags'] = user_tags_raw['tag']

In [16]:
user_genres_raw = general_df.groupby('userId')['genres'].apply(list).reset_index() #каждый фильм для user_id который он смотрел имеет список genres
user_genres_raw['user_all_genres'] = [list(set((itertools.chain(*user_genres_raw['genres'].iloc[i])))) for i in range(len(user_genres_raw))]
user_data = pd.merge(user_data, user_genres_raw[['user_all_genres','userId']], on="userId", how='left')

In [17]:
user_tag_5_raw = general_df[general_df['rating'] == 5].groupby('userId')['tag'].apply(set).apply(list).reset_index()\
        .rename({'userId':'userId','tag':'user_5_tags'}, axis='columns')#каждый user_id список tags когда оценка 5
user_data = pd.merge(user_data, user_tag_5_raw, on="userId", how='left')

In [18]:
user_genres_5_raw = general_df[(general_df['rating'] == 5)].groupby('userId')['genres'].apply(list).reset_index()
user_genres_5_raw['user_5_genres'] = [list(set((itertools.chain(*user_genres_5_raw['genres'].iloc[i])))) for i in range(len(user_genres_5_raw))]
user_data = pd.merge(user_data, user_genres_5_raw[['user_5_genres','userId']], on="userId", how='left')

In [19]:
user_tag_less3_raw = general_df[general_df['rating'] <= 3].groupby('userId')['tag'].apply(set).apply(list).reset_index()\
         .rename({'userId':'userId','tag':'user_less3_tags'}, axis='columns') #каждый user_id список tags когда оценка 4-4.5
user_data = pd.merge(user_data, user_tag_less3_raw, on="userId", how='left')

In [20]:
user_genres_less3_raw = general_df[general_df['rating'] <= 3].groupby('userId')['genres'].apply(list).reset_index() #каждый user_id список tags когда оценка 4-4.5
user_genres_less3_raw['user_less3_genres'] = [list(set((itertools.chain(*user_genres_less3_raw['genres'].iloc[i])))) for i in range(len(user_genres_less3_raw))]

In [21]:
user_data = pd.merge(user_data, user_genres_less3_raw[['user_less3_genres','userId']], on="userId", how='left')

Немного статестической информации о пользователе (в случае если информация отсутсвует присваем среднее значение в данном параметре по пользователям)

In [22]:
user_var = general_df.groupby('userId')['rating'].var().reset_index()
user_var = user_var.fillna(np.round(np.mean(user_var['rating']), 2))\
    .rename({'userId':'userId','rating':'user_var'}, axis='columns') 
user_mean = general_df.groupby('userId')['rating'].mean().reset_index()
user_mean = user_mean.fillna(np.round(np.mean(user_mean['rating']), 2))\
        .rename({'userId':'userId','rating':'user_mean'}, axis='columns') 
user_std = general_df.groupby('userId')['rating'].std().reset_index()
user_std = user_std.fillna(np.round(np.mean(user_std['rating']), 2))\
        .rename({'userId':'userId','rating':'user_std'}, axis='columns')

In [23]:
user_data = pd.merge(user_data, user_mean, on="userId", how='left')
user_data = pd.merge(user_data, user_std, on="userId", how='left')
user_data = pd.merge(user_data, user_var, on="userId", how='left')

In [24]:
user_data = user_data.fillna(' ')

In [25]:
user_data_test = user_data.drop(axis=1, labels=['userId','user_mean','user_std','user_var'])

In [26]:
list(set(user_data['userId']) - set(general_df['userId'])) # все ID СОВПАДАЮТ!

[]

In [27]:
user_data['user_text'] = user_data_test.apply(lambda x: ' ,'.join(x.dropna().astype(str)),axis=1)\
    .apply(lambda x: " ".join(sorted(list(set(x.replace('[','').replace(']','')\
    .replace('"','').replace(' ','').replace("'","").lower().split(','))))))

# Create movie data

In [28]:
general_df.head()

,movieId,title,genres,userId,tag,timestamp_x,rating,timestamp_y
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1040,animated,1514919574,2.5,1.514920e+09
1,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1040,buddy movie,1514919577,2.5,1.514920e+09
2,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1040,Cartoon,1514919589,2.5,1.514920e+09
3,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1040,cgi,1514919556,2.5,1.514920e+09
4,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1040,comedy,1514919569,2.5,1.514920e+09


In [29]:
movie_data = pd.DataFrame()

Какие теги пользователи ставили этому фильму?

In [30]:
movie_tags_raw = general_df.groupby('movieId')['tag'].apply(set).apply(list).reset_index() #каждый user_id имеет свой список tags
movie_data['movieId'] = movie_tags_raw['movieId']
movie_data['movie_all_tags'] = movie_tags_raw['tag']

Какие жанры существуют у данного фильма?

In [31]:
movie_genres_raw = general_df.groupby('movieId')['genres'].apply(list).reset_index() #каждый фильм для user_id который он смотрел имеет список genres
movie_genres_raw['movie_all_genres'] = [list(set((itertools.chain(*movie_genres_raw['genres'].iloc[i])))) for i in range(len(movie_genres_raw))]
movie_data = pd.merge(movie_data, movie_genres_raw[['movie_all_genres','movieId']], on="movieId", how='left')

Какие теги поставили, когда фильм был оценен на 5? (оно вообще тут нужно?)

In [32]:
movie_tag_5_raw = general_df[general_df['rating'] == 5].groupby('movieId')['tag'].apply(set).apply(list).reset_index()\
        .rename({'movieId':'movieId','tag':'movie_5_tags'}, axis='columns')#каждый user_id список tags когда оценка 5
movie_data = pd.merge(movie_data, movie_tag_5_raw, on="movieId", how='left')

Какие жанры поставили, когда фильм был оценен на 5? (оно вообще тут нужно?)

In [33]:
movie_genres_5_raw = general_df[(general_df['rating'] == 5)].groupby('movieId')['genres'].apply(list).reset_index()
movie_genres_5_raw['movie_5_genres'] = [list(set((itertools.chain(*movie_genres_5_raw['genres'].iloc[i])))) for i in range(len(movie_genres_5_raw))]
movie_data = pd.merge(movie_data, movie_genres_5_raw[['movie_5_genres','movieId']], on="movieId", how='left')

Какие теги поставили, когда фильм был оценен хуже или равный оценке 3? (Оно вообще тут нужно?)

In [34]:
movie_tag_less3_raw = general_df[general_df['rating'] <= 3].groupby('movieId')['tag'].apply(set).apply(list).reset_index()\
         .rename({'movieId':'movieId','tag':'movie_less3_tags'}, axis='columns') #каждый user_id список tags когда оценка 4-4.5
movie_data = pd.merge(movie_data, movie_tag_less3_raw, on="movieId", how='left')

Какие жанры поставили, когда фильм был оценен хуже или равный оценке 3? (Оно вообще тут нужно?)

In [35]:
movie_genres_less3_raw = general_df[general_df['rating'] <= 3].groupby('movieId')['genres'].apply(list).reset_index() #каждый user_id список tags когда оценка 4-4.5
movie_genres_less3_raw['movie_less3_genres'] = [list(set((itertools.chain(*movie_genres_less3_raw['genres'].iloc[i])))) for i in range(len(movie_genres_less3_raw))]

In [36]:
movie_data = pd.merge(movie_data, movie_genres_less3_raw[['movie_less3_genres','movieId']], on="movieId", how='left')

Немного статестической информации о фильме (в случае если информация отсутсвует присваем среднее значение в данном параметре по всем фильмам)

In [37]:
movie_var = general_df.groupby('movieId')['rating'].var().reset_index()
movie_var = movie_var.fillna(np.round(np.mean(movie_var['rating']), 2))\
    .rename({'movieId':'movieId','rating':'movie_var'}, axis='columns') 
movie_mean = general_df.groupby('movieId')['rating'].mean().reset_index()
movie_mean = movie_mean.fillna(np.round(np.mean(movie_mean['rating']), 2))\
        .rename({'movieId':'movieId','rating':'movie_mean'}, axis='columns') 
movie_std = general_df.groupby('movieId')['rating'].std().reset_index()
movie_std = movie_std.fillna(np.round(np.mean(movie_std['rating']), 2))\
        .rename({'movieId':'movieId','rating':'movie_std'}, axis='columns')

In [38]:
movie_data = pd.merge(movie_data, movie_mean, on="movieId", how='left')
movie_data = pd.merge(movie_data, movie_std, on="movieId", how='left')
movie_data = pd.merge(movie_data, movie_var, on="movieId", how='left')

In [39]:
movie_data = movie_data.fillna(' ')

In [40]:
movie_data.head()

,movieId,movie_all_tags,movie_all_genres,movie_5_tags,movie_5_genres,movie_less3_tags,movie_less3_genres,movie_mean,movie_std,movie_var
0,1,"[Cartoon, avi, rated-G, walkie talkie, lovable...","[Fantasy, Animation, Adventure, Comedy, Children]","[Buzz Lightyear, CGI, children cartoon, Cartoo...","[Fantasy, Animation, Adventure, Comedy, Children]","[animmation, Cartoon, Tom Hanks, cgi, jealousy...","[Fantasy, Animation, Adventure, Comedy, Children]",3.958728,1.023101,1.046736
1,2,"[Dynamic CGI Action, horrifying (but not Horro...","[Children, Fantasy, Adventure]","[board game, Children, fiction, based on a boo...","[Children, Fantasy, Adventure]","[Dynamic CGI Action, horrifying (but not Horro...","[Children, Fantasy, Adventure]",3.555556,1.077902,1.161873
2,3,"[Funniest Movies, comedy, good soundtrack, fun...","[Romance, Comedy]",[funny],"[Romance, Comedy]","[Ann Margaret, comedinha de velhinhos engraÃ§a...","[Romance, Comedy]",2.695652,0.926125,0.857708
3,4,"[interracial relationship, revenge, based on n...","[Drama, Romance, Comedy]",,,"[characters, revenge, chick flick]","[Drama, Romance, Comedy]",2.800000,1.036822,1.075000
4,5,"[midlife crisis, baby, watched under duress, p...",[Comedy],,,"[Fantasy, steve martin, sequel fever, remake, ...",[Comedy],2.920000,1.433818,2.055833


In [41]:
movie_data_test = movie_data.drop(axis=1, labels=['movieId','movie_mean','movie_std','movie_var'])

Создаем общий текст на основе полученных данных по фильмам

In [42]:
movie_data['movie_text'] = movie_data_test.apply(lambda x: ' ,'.join(x.dropna().astype(str)),axis=1)\
    .apply(lambda x: " ".join(sorted(list(set(x.replace('[','').replace(']','')\
    .replace('"','').replace(' ','').replace("'","").lower().split(','))))))

In [43]:
set(movie_data['movieId']) - set(general_df['movieId']) # все ID СОВПАДАЮТ!

set()

# Create one dataset for users | films dataframes via CROSS JOIN

In [67]:
small_movie = movie_data.iloc[:500]

In [68]:
small_user = user_data.iloc[:500]

In [69]:
small_all_raw_df = small_movie.merge(small_user, how='cross')

In [70]:
df = small_all_raw_df.merge(rating_df[['movieId','userId','rating']], how='inner', on=['movieId','userId'])

In [71]:
len(df)

9648

In [72]:
df.to_csv('dask_raw_df/df_raw_pandas.csv') 